In [3]:
import pandas as pd
import csv
import random

In [4]:

df = pd.read_excel('daten.xlsx', sheet_name=None)

In [5]:
print(df)

{'PLZ':       PLZ         Stadtteil      Lage
0   20095  Hamburg-Altstadt  sehr gut
1   20097         St. Georg       gut
2   20099  Hamburg-Altstadt  sehr gut
3   20144        Eimsbüttel       gut
4   20146      Harvestehude  sehr gut
..    ...               ...       ...
95  22761        Bahrenfeld       gut
96  22763          Ottensen  sehr gut
97  22765      Sternschanze  sehr gut
98  22767         St. Pauli       gut
99  22769         St. Pauli       gut

[100 rows x 3 columns], 'Zimmer':     m2  Zimmer
0   39       1
1   58       2
2   87       3
3  124       4
4  160       5
5  222       6, 'Energie':   Energie  Start  Ende
0       A   2016  2023
1       B   2010  2015
2       C   1993  2009
3       D   1973  1992
4       E   1966  1972
5       F   1956  1965
6       G   1910  1955, 'Miete':    Index  Miete von  Miete bis    Lage
0     C1       9.06      14.81  normal
1     C2       8.50      13.22  normal
2     C3       8.80      12.88  normal
3     C4       8.21      11.78  no

In [6]:
def getM2IndexFromLocation(m2, df, location):
    m2_mask = (df['m2 von'] <= m2) & (m2 <= df['m2 bis'])
    lage_mask = df['Lage'] == location

    # Apply the masks to select the desired row(s)
    selected_row = df[m2_mask & lage_mask]
    return selected_row["Index"].values[0]

def getAgeIndex(age, df):
    age_mask = (df['BJ von'] <= age) & (age <= df['BJ Bis'])
    selected_row = df[age_mask]
    return selected_row["Index"].values[0]

def getRooms(m2, df):
    m2_mask = (m2 <= df['m2'])
    selected_row = df[m2_mask]
    return selected_row["Zimmer"].values[0]

def getEnergie(age, df):
    age_mask = (df['Start'] <= age) & (age <= df['Ende'])
    selected_row = df[age_mask]
    return selected_row["Energie"].values[0]

def getRent(key, df):
    key_mask = (df['Index'] == key)
    selected_row = df[key_mask]
    rent = round(random.uniform(selected_row["Miete von"], selected_row["Miete bis"]), 2)
    return rent.values[0]

def getDistrictAndLocation(plz, df):
    key_mask = (df['PLZ'] == plz)
    selected_row = df[key_mask]
    district = selected_row["Stadtteil"].values[0]
    location = selected_row["Lage"].values[0]
    if location == "sehr gut":
        location = "gut"
    return ( district,  location)
    
def randomData(plz, district, location, df):
    l = random.randint(1, 200) #Wie viel Objekte fuer eine Plz erstellt werden
    result = []
    for _ in range(l):
        row = [plz, district, location]
        age = random.randint(1910, 2023)
        row.append(age)
        m2 = random.randint(15, 222)
        row.append(m2)
        m2Index  = getM2IndexFromLocation(m2, df["M2"], location)
        ageIndex = getAgeIndex(age, df["Baujahr"])
        rooms = getRooms(m2, df["Zimmer"])
        row.append(rooms)
        energie = getEnergie(age, df["Energie"])
        row.append(energie)
        key = ageIndex + str(m2Index)
        rent = getRent(key, df["Miete"])
        row.append(rent)
        result.append(row)
    return result

header = ['zipcode', 'district', 'location', 'age', 'm2', 'rooms', 'energie', 'rent']

def writeToFile(result):
    with open('data.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(result)

    print('CSV file created successfully.')

In [7]:

plzs = df["PLZ"]["PLZ"]
result = []
for plz in plzs:
    (district, location) = getDistrictAndLocation(plz, df["PLZ"])
    result.append(randomData(plz, district, location, df))
    
result = [item for sublist in result for item in sublist] #Result Listen in eine Liste konkatenieren
    
writeToFile(result)

CSV file created successfully.
